# Import

In [3]:
from ray.rllib.examples.policy.rock_paper_scissors_dummies import (
    BeatLastHeuristic,
    AlwaysSameHeuristic,
)
import os
import random
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from gymnasium.spaces import Tuple, Discrete, Box,Dict
import ray
from ray import air
from ray import tune
from ray.tune.registry import register_env
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import numpy as np
from ray.rllib.utils import check_env
from typing import Optional
from ray.rllib.algorithms.pg import (
    PG,
    PGConfig,
    PGTF2Policy,
    PGTF1Policy,
    PGTorchPolicy,
)
import argparse
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.algorithms.maddpg.maddpg import MADDPGConfig,DQN
from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector


In [56]:
import numpy as np
#import rlcard
from gymnasium import spaces
from pettingzoo.utils import agent_selector
from pettingzoo import AECEnv


class MyEnv(AECEnv):
    def __init__(self):
        super().__init__()
        """ self.name = 'tesi'
        self.num_players = 2
        config = {
            "allow_step_back": False,
            "seed": None,
            "game_num_players": self.num_players,
        } """

        #self.env = rlcard.make(self.name, config)
        self.agents =['attaccante','difensore']
        self.possible_agents = self.agents[:]

        """ dtype = self.env.reset()[0]["obs"].dtype
        if dtype == np.dtype(np.int64):
            self._dtype = np.dtype(np.int8)
        elif dtype == np.dtype(np.float64):
            self._dtype = np.dtype(np.float32)
        else:
            self._dtype = dtype """

        self.observation_spaces = self._convert_to_dict(
            [
                spaces.Dict(
                    {
                        "observation": spaces.Box(
                            low=0.0, high=1.0, shape=(2,), dtype=bool
                        ),
                    }
                )
                for _ in range(self.num_agents)
            ]
        )
        self.action_spaces = self._convert_to_dict(
            [spaces.Discrete(3) for _ in range(self.num_agents)]
        )

    def observation_space(self, agent):
        return self.observation_spaces[agent]

    def action_space(self, agent):
        return self.action_spaces[agent]

    def _scale_rewards(self, reward):
        return reward

    def _int_to_name(self, ind):
        return self.possible_agents[ind]

    def _name_to_int(self, name):
        return self.possible_agents.index(name)

    def _convert_to_dict(self, list_of_list):
        return dict(zip(self.possible_agents, list_of_list))

    def observe(self, agent):
        #obs = self.env.get_state(self._name_to_int(agent))
        #observation = obs["obs"].astype(self._dtype)

        #legal_moves = self.next_legal_moves
        #action_mask = np.zeros(self.env.num_actions, "int8")
        """ for i in legal_moves:
            action_mask[i] = 1 """

        return {"observation": self.observation_spaces[agent]}

    def step(self, action):
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            return self._was_dead_step(action)
        #obs, next_player_id = self.env.step(action)
        #next_player = self._int_to_name(next_player_id)
        #self._last_obs = self.observe(self.agent_selection)
        """ if self.env.is_over():
            self.rewards = self._convert_to_dict(
                self._scale_rewards(self.env.get_payoffs())
            )
            self.next_legal_moves = []
            self.terminations = self._convert_to_dict(
                [True for _ in range(self.num_agents)]
            )
            self.truncations = self._convert_to_dict(
                [False for _ in range(self.num_agents)]
            ) """
        #else:
            #self.next_legal_moves = obs["legal_actions"]
        #self._cumulative_rewards[self.agent_selection] = 0
        #self.agent_selection = next_player
        self._accumulate_rewards()
        self._deads_step_first()

    def reset(self, seed=None, options=None):
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.reset()
        """ if seed is not None:
            self._seed(seed=seed) """
        #obs, player_id = self.env.reset()
        #self.agents = self.possible_agents[:]
        #self.agent_selection = self._int_to_name(player_id)
        self.rewards = self._convert_to_dict([0 for _ in range(self.num_agents)])
        self._cumulative_rewards = self._convert_to_dict(
            [0 for _ in range(self.num_agents)]
        )
        self.terminations = self._convert_to_dict(
            [False for _ in range(self.num_agents)]
        )
        self.truncations = self._convert_to_dict(
            [False for _ in range(self.num_agents)]
        )
        """ self.infos = self._convert_to_dict(
            [{"legal_moves": []} for _ in range(self.num_agents)]
        ) """
        """ self.next_legal_moves = list(sorted(obs["legal_actions"]))
        self._last_obs = obs["obs"] """

    def render(self):
        raise NotImplementedError()

    def close(self):
        pass

In [41]:
class TestEnv(MultiAgentEnv):
#class TestEnv(AECEnv):
    def __init__(self):
        super().__init__()
        # Mosse 
        self.MOVES = ['mossa1', 'mossa2','mossa3']
        # Questa è la truncation cosi esce per non girare all'infinito
        self.NUM_ITERS = 20
        # Mappa che in base all'azione eseguita mi da costo, impatto, ecc dell'azione
        self.REWARD_MAP = {
            'mossa1': (1, 0, 0),
            'mossa2': (2, 0, 0),
            'mossa3': (3, 0, 0)
        }
        # per la funzione di reward
        self.wt = 0.5
        self.wc = 0.5
        self.wi = 0.5
        self.tMax = 100
        self.cMax = 100
        self.agents = ['attaccante','difensore']
        self.agent_ids = set(self.agents)
        """ self.observation_spaces = {
            agent:Dict(
                {
                    'observation': Box(low=0,high=1,shape=(2,),dtype=np.float32),
                }
            )
            for agent in self.agents
        } """
        self.observation_spaces = self._convert_to_dict(
            [
                Dict(
                    {
                        "observation": Box(
                            low=0.0, high=1.0, shape=(2,), dtype=np.dtype(np.float32)
                        ),
                    }
                )
                for _ in self.agents
            ]
        )
        #self.action_spaces = {agent: Discrete(3) for agent in self.agents}
        self.action_spaces = self._convert_to_dict(
            [Discrete(3) for _ in self.agents]
        )

    def _convert_to_dict(self, list_of_list):
        return dict(zip(self.agents, list_of_list))
    
    def step(self,allAction):
        """ if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            #self._was_dead_step(action)
            print('ECCOOOO:',self.agents)
            return """

        for agent in self.agents:
            #agent = self.agent_selection
            print('Agente in azione:',agent)
            print('allAction:',allAction)
            """ action = allAction[agent]
            reward = self.REWARD_MAP[self.MOVES[action]]
            valReward = -self.wt*(reward[0]/self.tMax)-self.wc*(reward[1]/self.cMax)-self.wi*(1) 
            #rewardInv = -valReward
            print('Reward:',valReward)
            if agent == 'attaccante':
                self.rewards[self.agents[0]], self.rewards[self.agents[1]] = (valReward,0)
            else:
                self.rewards[self.agents[0]], self.rewards[self.agents[1]] = (0,valReward) """
            self.num_moves += 1
            # The truncations dictionary must be updated for all players.
            self.truncations = {
                agent: self.num_moves >= self.NUM_ITERS for agent in self.agents
            }
            self.agent_selection = self._agent_selector.next()
            # Adds .rewards to ._cumulative_rewards
            #self._accumulate_rewards() 
        return self.observation_spaces,self.rewards,self.terminations,self.truncations,self.infos
        

        
    def reset(self,seed=None,options=None):
        #super().reset()
        self.agents = self.agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}      
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        # credo serve per arrestare
        self.num_moves = 0
        
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.reset()
        
        return self.observation_spaces,self.infos
    
    def observation_space(self, agent):
        return self.observation_spaces[agent]
    
    def action_space(self, agent):
        return self.action_spaces[agent]
    def observe(self, agent):
        return {"observation": self.observation_space(agent)}
        


def env_creator(args):
    env = TestEnv(args)

    obs_space = {agent: env.observation_space(agent)['observation'] for agent in env.agents}
    act_space = {agent: env.action_space(agent) for agent in env.agents}
    print('obs_space:',obs_space)
    print('act_space:',act_space)

    #n_agents = len(args["agents"])
    grouping = {'group1':env.agents}
    return env.with_agent_groups( grouping,obs_space=obs_space, act_space=act_space)

ray.shutdown()
ray.init(num_gpus=1)

register_env("grouped_test", env_creator)

tf1, tf, tfv = try_import_tf()
torch, _ = try_import_torch()


2023-09-27 14:37:57,393	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [10]:
def select_policy(agent_id, episode, **kwargs):
        return agent_id
        
stop = {
        "training_iteration": 3,
        "timesteps_total": 100000,
        "episode_reward_mean": 1000.0,
    }

In [163]:
config = PGConfig().environment("grouped_test",disable_env_checking=True).framework("torch").multi_agent(
            policies={
                "group1": PolicySpec(policy_class=AlwaysSameHeuristic,
                                          observation_space=Box(low=0,high=1,shape=(2,),dtype=np.float32),
                                          action_space=Discrete(3)),
            },
            policy_mapping_fn=select_policy,
            policies_to_train=["group1"],
        )
results = tune.Tuner(
        "PG", param_space=config, run_config=air.RunConfig(stop=stop, verbose=1)
    ).fit()
print(results)

(pid=541051) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PG pid=541051) 2023-09-27 11:31:53,139	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/pg/` has been deprecated. Use `rllib_contrib/pg/` instead. This will raise an error in the future!
(PG pid=541051) 2023-09-27 11:31:53,140	WARNING algorithm_config.py:672 -- Cannot create PGConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-09-27 11:31:53,489	ERROR tune_controller.py:1502 -- Trial task failed for trial PG_grouped_test_ae2e4_00000
Traceback (most recent call last):
  File "/home/matteo/.local/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/matteo/.local/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/matteo/.local/lib/python3.10/site-packag

(PG pid=541051) obs_space: {'attaccante': Box(0.0, 1.0, (2,), float32), 'difensore': Box(0.0, 1.0, (2,), float32)}
(PG pid=541051) act_space: {'attaccante': Discrete(3), 'difensore': Discrete(3)}


2023-09-27 11:31:54,045	ERROR tune.py:1139 -- Trials did not complete: [PG_grouped_test_ae2e4_00000]
2023-09-27 11:31:54,047	INFO tune.py:1143 -- Total run time: 6.26 seconds (5.69 seconds for the tuning loop).


ResultGrid<[
  Result(
    error='RayTaskError(TypeError)',
    metrics={},
    path='/home/matteo/ray_results/PG_2023-09-27_11-31-47/PG_grouped_test_ae2e4_00000_0_2023-09-27_11-31-47',
    filesystem='local',
    checkpoint=None
  )
]>


In [230]:
config = MADDPGConfig().environment("grouped_test").framework("torch").multi_agent(
            policies={
                "group1": PolicySpec(policy_class=AlwaysSameHeuristic,
                                          observation_space=Box(low=0,high=1,shape=(2,),dtype=np.float32),
                                          action_space=Discrete(3)),
            },
            policy_mapping_fn=select_policy,
            policies_to_train=["group1"],
        )
results = tune.Tuner(
        "MADDPG", param_space=config, run_config=air.RunConfig(stop=stop, verbose=1)
    ).fit()
print(results)

(pid=631750) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=631750) 2023-09-27 12:46:12,455	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
(pid=631750) 2023-09-27 12:46:12,456	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
(MADDPG pid=631750) 2023-09-27 12:46:12,490	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/maddpg/` has been deprecated. Use `rllib_contrib/maddpg/` instead. This will raise an error in the future!
(MADDPG pid=631750) 2023-09-27 12:46:12,490	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
(MADDPG pid=631750) 2023-09-27 12:46:12,490	WARNING algorithm_config.py:672 -- Cannot create MADDPGConfig from 

ResultGrid<[
  Result(
    error='TuneError',
    metrics={},
    path='/home/matteo/ray_results/MADDPG_2023-09-27_12-46-07/MADDPG_grouped_test_1031f_00000_0_2023-09-27_12-46-07',
    filesystem='local',
    checkpoint=None
  )
]>


In [17]:
import numpy as np
import rlcard
from gymnasium import spaces

from pettingzoo import AECEnv


class RLCardBase(AECEnv):
    def __init__(self, name: str, num_players: int, obs_shape: tuple):
        super().__init__()
        self.name = name
        self.num_players = num_players
        config = {
            "allow_step_back": False,
            "seed": None,
            "game_num_players": num_players,
        }

        self.env = rlcard.make(name, config)
        self.screen = None
        if not hasattr(self, "agents"):
            self.agents = [f"player_{i}" for i in range(num_players)]
        self.possible_agents = self.agents[:]

        dtype = self.env.reset()[0]["obs"].dtype
        if dtype == np.dtype(np.int64):
            self._dtype = np.dtype(np.int8)
        elif dtype == np.dtype(np.float64):
            self._dtype = np.dtype(np.float32)
        else:
            self._dtype = dtype

        self.observation_spaces = self._convert_to_dict(
            [
                spaces.Dict(
                    {
                        "observation": spaces.Box(
                            low=0.0, high=1.0, shape=obs_shape, dtype=self._dtype
                        ),
                        "action_mask": spaces.Box(
                            low=0, high=1, shape=(self.env.num_actions,), dtype=np.int8
                        ),
                    }
                )
                for _ in range(self.num_agents)
            ]
        )
        self.action_spaces = self._convert_to_dict(
            [spaces.Discrete(self.env.num_actions) for _ in range(self.num_agents)]
        )

    def observation_space(self, agent):
        return self.observation_spaces[agent]

    def action_space(self, agent):
        return self.action_spaces[agent]

    def _seed(self, seed=None):
        config = {
            "allow_step_back": False,
            "seed": seed,
            "game_num_players": self.num_players,
        }
        self.env = rlcard.make(self.name, config)

    def _scale_rewards(self, reward):
        return reward

    def _int_to_name(self, ind):
        return self.possible_agents[ind]

    def _name_to_int(self, name):
        return self.possible_agents.index(name)

    def _convert_to_dict(self, list_of_list):
        return dict(zip(self.possible_agents, list_of_list))

    def observe(self, agent):
        obs = self.env.get_state(self._name_to_int(agent))
        observation = obs["obs"].astype(self._dtype)

        legal_moves = self.next_legal_moves
        action_mask = np.zeros(self.env.num_actions, "int8")
        for i in legal_moves:
            action_mask[i] = 1

        return {"observation": observation, "action_mask": action_mask}

    def step(self, action):
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            return self._was_dead_step(action)
        obs, next_player_id = self.env.step(action)
        next_player = self._int_to_name(next_player_id)
        self._last_obs = self.observe(self.agent_selection)
        if self.env.is_over():
            self.rewards = self._convert_to_dict(
                self._scale_rewards(self.env.get_payoffs())
            )
            self.next_legal_moves = []
            self.terminations = self._convert_to_dict(
                [True for _ in range(self.num_agents)]
            )
            self.truncations = self._convert_to_dict(
                [False for _ in range(self.num_agents)]
            )
        else:
            self.next_legal_moves = obs["legal_actions"]
        
        self._cumulative_rewards[self.agent_selection] = 0
        self.agent_selection = next_player
        self._accumulate_rewards()
        self._deads_step_first()

    def reset(self, seed=None, options=None):
        if seed is not None:
            self._seed(seed=seed)
        obs, player_id = self.env.reset()
        self.agents = self.possible_agents[:]
        self.agent_selection = self._int_to_name(player_id)
        self.rewards = self._convert_to_dict([0 for _ in range(self.num_agents)])
        self._cumulative_rewards = self._convert_to_dict(
            [0 for _ in range(self.num_agents)]
        )
        self.terminations = self._convert_to_dict(
            [False for _ in range(self.num_agents)]
        )
        self.truncations = self._convert_to_dict(
            [False for _ in range(self.num_agents)]
        )
        self.infos = self._convert_to_dict(
            [{"legal_moves": []} for _ in range(self.num_agents)]
        )
        self.next_legal_moves = list(sorted(obs["legal_actions"]))
        self._last_obs = obs["obs"]

    def render(self):
        raise NotImplementedError()

    def close(self):
        pass

In [18]:
import os

import ray
from gymnasium.spaces import Box, Discrete
from ray import tune
from ray.rllib.algorithms.dqn import DQNConfig
from ray.rllib.algorithms.dqn.dqn_torch_model import DQNTorchModel
from ray.rllib.env import PettingZooEnv
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.torch_utils import FLOAT_MAX
from ray.tune.registry import register_env
import rsp

#from pettingzoo.classic import leduc_holdem_v4

torch, nn = try_import_torch()


class TorchMaskedActions(DQNTorchModel):
    """PyTorch version of above ParametricActionsModel."""

    def __init__(
        self,
        obs_space: Box,
        action_space: Discrete,
        num_outputs,
        model_config,
        name,
        **kw,
    ):
        DQNTorchModel.__init__(
            self, obs_space, action_space, num_outputs, model_config, name, **kw
        )

        obs_len = obs_space.shape[0] - action_space.n

        orig_obs_space = Box(
            shape=(obs_len,), low=obs_space.low[:obs_len], high=obs_space.high[:obs_len]
        )
        self.action_embed_model = TorchFC(
            orig_obs_space,
            action_space,
            action_space.n,
            model_config,
            name + "_action_embed",
        )

    def forward(self, input_dict, state, seq_lens):
        # Extract the available actions tensor from the observation.
        action_mask = input_dict["obs"]["action_mask"]

        # Compute the predicted action embedding
        action_logits, _ = self.action_embed_model(
            {"obs": input_dict["obs"]["observation"]}
        )
        # turns probit action mask into logit action mask
        inf_mask = torch.clamp(torch.log(action_mask), -1e10, FLOAT_MAX)

        return action_logits + inf_mask, state

    def value_function(self):
        return self.action_embed_model.value_function()


if __name__ == "__main__":
    #ray.init()

    alg_name = "DQN"
    ModelCatalog.register_custom_model("pa_model", TorchMaskedActions)
    # function that outputs the environment you wish to register.

    def env_creator():
        env = RLCardBase("leduc-holdem", 2, (36,))
        #env = rsp.env(render_mode="human")
        #env = TestEnv()
        #env = MyEnv()
        return env

    env_name = "leduc_holdem_v4"
    register_env(env_name, lambda config: PettingZooEnv(env_creator()))

    test_env = PettingZooEnv(env_creator())
    obs_space = test_env.observation_space
    act_space = test_env.action_space

    config = PGConfig().environment(env_name,disable_env_checking=True).framework("torch").multi_agent(
            policies={
                "player_0": (None, obs_space, act_space, {}),
                "player_1": (None, obs_space, act_space, {}),
            },
            policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
        )
    results = tune.Tuner(
            "PG", param_space=config, run_config=air.RunConfig(stop=stop, verbose=1)
        ).fit()
    print(results)
    
    """ config = (
        DQNConfig()
        .environment(env=env_name)
        .rollouts(num_rollout_workers=1, rollout_fragment_length=30)
        .training(
            train_batch_size=200,
            hiddens=[],
            dueling=False,
            model={"custom_model": "pa_model"},
        )
        .multi_agent(
            policies={
                "player_0": (None, obs_space, act_space, {}),
                "player_1": (None, obs_space, act_space, {}),
            },
            policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
        )
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
        .debugging(
            log_level="DEBUG"
        )  # TODO: change to ERROR to match pistonball example
        .framework(framework="torch")
        .exploration(
            exploration_config={
                # The Exploration class to use.
                "type": "EpsilonGreedy",
                # Config for the Exploration class' constructor:
                "initial_epsilon": 0.1,
                "final_epsilon": 0.0,
                "epsilon_timesteps": 100000,  # Timesteps over which to anneal epsilon.
            }
        )
    )

    tune.run(
        alg_name,
        name="DQN",
        stop={"timesteps_total": 10000000},
        checkpoint_freq=10,
        config=config.to_dict(),
    ) """

(pid=21342) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PG pid=21342) 2023-09-28 09:34:00,372	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/pg/` has been deprecated. Use `rllib_contrib/pg/` instead. This will raise an error in the future!
(PG pid=21342) 2023-09-28 09:34:00,372	WARNING algorithm_config.py:672 -- Cannot create PGConfig from given `config_dict`! Property __stdout_file__ not supported.


(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'p

(PG pid=21342) 2023-09-28 09:34:00,706	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.complex_input_net.ComplexInputNetwork` has been deprecated. This will raise an error in the future!
(PG pid=21342) 2023-09-28 09:34:00,706	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(PG pid=21342) 2023-09-28 09:34:00,707	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(PG pid=21342) 2023-09-28 09:34:00,716	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
(PG pid=21342) 2023-09-28 09:34:00,720	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_action_dist.TorchDistributionWrapper

(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'p

(PG pid=21342) 2023-09-28 09:34:01,398	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.train_one_step` has been deprecated. This will raise an error in the future!


(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'player_1': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32))}
(PG pid=21342) {'player_0': Dict('action_mask': Box(0, 1, (4,), int8), 'observation': Box(0.0, 1.0, (36,), float32)), 'p

(PG pid=21342) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/matteo/ray_results/PG_2023-09-28_09-33-55/PG_leduc_holdem_v4_61442_00000_0_2023-09-28_09-33-55/checkpoint_000000)
2023-09-28 09:34:03,564	INFO tune.py:1143 -- Total run time: 7.92 seconds (7.74 seconds for the tuning loop).


ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'player_0': {'learner_stats': {'allreduce_latency': 0.0, 'policy_loss': -0.8583216071128845, 'cur_lr': 0.0004}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 96, 'num_grad_updates_lifetime': 3, 'diff_num_grad_updates_vs_sampler_policy': 0.0}, 'player_1': {'learner_stats': {'allreduce_latency': 0.0, 'policy_loss': -0.007498854771256447, 'cur_lr': 0.0004}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 105, 'num_grad_updates_lifetime': 3, 'diff_num_grad_updates_vs_sampler_policy': 0.0}}, 'num_env_steps_sampled': 600, 'num_env_steps_trained': 600, 'num_agent_steps_sampled': 600, 'num_agent_steps_trained': 600}, 'sampler_results': {'episode_reward_max': 0.0, 'episode_reward_min': -0.5, 'episode_reward_mean': -0.215, 'episode_len_mean': 2.34, 'episode_media': {}, 'episodes_this_iter': 90, 'policy_reward_min': {'player_1': -4.0, 'player_0': -6.0}, 'policy_reward_

In [26]:
l = [False,True,True]
print(True if all(l)==True else False)
l = {'ciao':[0,1,2]}
print(l['ciao'][0])
l['pipoo'] = 0
print(l)
l['ciao'] = 4
print(l)

False
0
{'ciao': [0, 1, 2], 'pipoo': 0}
{'ciao': 4, 'pipoo': 0}


2023-09-28 12:42:21,148	WARNING worker.py:2058 -- The autoscaler failed with the following error:
Terminated with signal 15
  File "/home/matteo/.local/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 711, in <module>
    monitor.run()
  File "/home/matteo/.local/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 586, in run
    self._run()
  File "/home/matteo/.local/lib/python3.10/site-packages/ray/autoscaler/_private/monitor.py", line 440, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)



: 

In [33]:
import numpy as np
l = [1,1,1]
print(np.stack([True,True,False]).dtype)
print(np.ndarray(shape=(len(l)),buffer=np.array(l),dtype=np.int))

bool
[1 1 1]


/tmp/ipykernel_9957/2774214198.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(np.ndarray(shape=(len(l)),buffer=np.array(l),dtype=np.int))


In [4]:
p = {

}
print(p)
p['ciao'] = 0
print(p)
p['Hole'] = 0
print(p)

{}
{'ciao': 0}
{'ciao': 0, 'Hole': 0}


In [6]:
l= [False,False]
for i in range(len(l)):
    l[i] = not(l[i])
print(l)

[True, True]


In [10]:
l = ['pippo','pluto']
n = 'pippo'
print(l[1-l.index(n)])

pippo


In [4]:
er = [345, 249, 30, 30, 102, 168, 135, 306, 294, 45, 39, 57, 51, 42, 60, 30, 81, 534, 426, 348, 180, 201, 54, 69, 39, 30, 30, 903, 30, 111, 453, 30, 30, 117, 366, 339, 30, 30, 192, 99, 30, 363, 72, 393, 222, 30, 678, 36, 30, 30, 42, 624, 36, 108, 75, 30, 201, 30, 513, 60, 63, 63, 39, 30, 39, 318, 30, 30, 78, 165, 390, 102, 108, 294, 45, 54, 30, 249, 282, 639, 42, 348, 126, 348, 81, 501, 48, 318, 663]
el = [45, 29, 3, 3, 17, 29, 23, 33, 39, 9, 5, 11, 9, 7, 9, 1, 13, 45, 45, 39, 33, 27, 9, 17, 7, 3, 3, 67, 7, 21, 45, 1, 1, 21, 41, 40, 1, 3, 33, 17, 3, 51, 15, 43, 43, 3, 71, 5, 5, 7, 9, 55, 7, 21, 15, 3, 33, 3, 59, 9, 15, 11, 7, 1, 5, 37, 5, 1, 15, 25, 47, 19, 23, 41, 7, 17, 1, 37, 37, 59, 9, 45, 21, 49, 17, 45, 7, 41, 69]
print(len(er))
print(len(el))

([345, 249, 30, 30, 102, 168, 135, 306, 294, 45, 39, 57, 51, 42, 60, 30, 81, 534, 426, 348, 180, 201, 54, 69, 39, 30, 30, 903, 30, 111, 453, 30, 30, 117, 366, 339, 30, 30, 192, 99, 30, 363, 72, 393, 222, 30, 678, 36, 30, 30, 42, 624, 36, 108, 75, 30, 201, 30, 513, 60, 63, 63, 39, 30, 39, 318, 30, 30, 78, 165, 390, 102, 108, 294, 45, 54, 30, 249, 282, 639, 42, 348, 126, 348, 81, 501, 48, 318, 663],)
